#Medallion Architecture pipeline (Bronze → Silver → Gold)

##Download & Store CSV in DBFS

In [0]:
import requests
csv_url = "https://raw.githubusercontent.com/prasertcbs/basic-dataset/master/Restaurant%20customer%20data.csv"
local_path = "/tmp/restaurant_data.csv"
dbfs_path = "dbfs:/mnt/raw_data/restaurant_data.csv"

# Download CSV file
response = requests.get(csv_url)
with open(local_path, "wb") as f:
    f.write(response.content)

# Move file to DBFS
dbutils.fs.mv(f"file://{local_path}", dbfs_path)

print("✅ File successfully stored in DBFS")

✅ File successfully stored in DBFS


##Ingest Data into the Bronze Layer (Raw Data)

In [0]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("AutoLoaderMedallion").getOrCreate()

# Load raw data using Auto Loader
df_bronze = spark.readStream.format("cloudFiles") \
    .option("cloudFiles.format", "csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("cloudFiles.schemaLocation", "dbfs:/mnt/schema/") \
    .load("dbfs:/mnt/raw_data/")

# Save to Delta Lake (Bronze Table)
df_bronze.writeStream.format("delta") \
    .option("checkpointLocation", "dbfs:/mnt/checkpoints/bronze/") \
    .start("dbfs:/mnt/bronze_table")

print("✅ Data loaded into Bronze layer")


✅ Data loaded into Bronze layer


In [0]:
display(df_bronze)

userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height,_rescued_data
U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77,null
U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87,null
U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69,null
U1004,18.867,-99.183,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53,null
U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69,null
U1006,22.15,-100.983,true,social drinker,no preference,friends,car owner,single,independent,1989,variety,hard-worker,none,student,blue,75,medium,1.8,null
U1007,22.118464,-100.938256,false,casual drinker,informal,solitary,public,single,independent,1989,variety,thrifty-protector,Catholic,student,purple,60,low,1.59,null
U1008,22.122989,-100.923811,false,social drinker,formal,solitary,public,single,independent,1989,technology,hard-worker,Catholic,student,green,68,low,1.72,null
U1009,22.159427,-100.990448,false,abstemious,formal,family,on foot,single,kids,1991,variety,thrifty-protector,Catholic,student,green,75,medium,1.78,null
U1010,22.190889,-100.998669,false,social drinker,no preference,friends,car owner,married,kids,1987,technology,hard-worker,none,student,green,40,medium,1.67,null


##Clean Data & Move to Silver Layer

In [0]:
from pyspark.sql.functions import col
from delta.tables import DeltaTable

# Load Bronze Table
df_bronze = spark.read.format("delta").load("dbfs:/mnt/bronze_table")

# Data Cleaning: Remove nulls, duplicates, and enforce schema
df_silver = df_bronze.dropDuplicates().filter(col("dress_preference").isNotNull())


# Save to Delta Lake (Silver Layer)
df_silver.write.format("delta").mode("overwrite").save("dbfs:/mnt/silver_table")

print("✅ Data cleaned and stored in Silver layer")


✅ Data cleaned and stored in Silver layer


In [0]:
print("🔹 Silver Data:")
spark.read.format("delta").load("dbfs:/mnt/silver_table").show()

🔹 Silver Data:
+------+---------+-----------+------+--------------+----------------+--------+---------+--------------+-----------+----------+------------+-------------------+---------+------------+------+------+------+------+-------------+
|userID| latitude|  longitude|smoker|   drink_level|dress_preference|ambience|transport|marital_status|      hijos|birth_year|    interest|        personality| religion|    activity| color|weight|budget|height|_rescued_data|
+------+---------+-----------+------+--------------+----------------+--------+---------+--------------+-----------+----------+------------+-------------------+---------+------------+------+------+------+------+-------------+
| U1004|   18.867|    -99.183| false|    abstemious|        informal|  family|   public|        single|independent|      1940|     variety|        hard-worker|     none|professional| green|    44|medium|  1.53|         null|
| U1046|22.144415|-100.933097| false|social drinker|   no preference| friends|car own

#Aggregate & Store in Gold Layer

In [0]:
df_silver = spark.read.format("delta").load("dbfs:/mnt/silver_table")

# Print actual column names
print(df_silver.columns)


['userID', 'latitude', 'longitude', 'smoker', 'drink_level', 'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos', 'birth_year', 'interest', 'personality', 'religion', 'activity', 'color', 'weight', 'budget', 'height', '_rescued_data']


###Rename Columns (Fix Special Characters)

In [0]:
from pyspark.sql.functions import col

# Rename Columns: Replace ?, Spaces & Special Characters
new_columns = [c.encode("ascii", "ignore").decode() for c in df_silver.columns]  # Remove non-ASCII characters
new_columns = [c.replace(" ", "_").replace("(", "").replace(")", "").replace("?", "").replace("-", "_") for c in new_columns]

# Apply New Column Names
df_silver = df_silver.toDF(*new_columns)

# Print New Column Names to Verify
print(df_silver.columns)


['userID', 'latitude', 'longitude', 'smoker', 'drink_level', 'dress_preference', 'ambience', 'transport', 'marital_status', 'hijos', 'birth_year', 'interest', 'personality', 'religion', 'activity', 'color', 'weight', 'budget', 'height', '_rescued_data']


###Convert Columns to Numeric Types (Default they are in strings)

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

# Convert numeric columns from string to appropriate types
df_silver = df_silver.withColumn("birth_year", col("birth_year").cast(IntegerType())) \
                     .withColumn("weight", col("weight").cast(FloatType())) \
                     .withColumn("budget", col("budget").cast(FloatType())) \
                     .withColumn("height", col("height").cast(FloatType()))


In [0]:
df_silver.printSchema()
df_silver.select("birth_year", "weight", "budget", "height").show(5)

root
 |-- userID: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- drink_level: string (nullable = true)
 |-- dress_preference: string (nullable = true)
 |-- ambience: string (nullable = true)
 |-- transport: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- hijos: string (nullable = true)
 |-- birth_year: integer (nullable = true)
 |-- interest: string (nullable = true)
 |-- personality: string (nullable = true)
 |-- religion: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- color: string (nullable = true)
 |-- weight: float (nullable = true)
 |-- budget: float (nullable = true)
 |-- height: float (nullable = true)
 |-- _rescued_data: string (nullable = true)

+----------+------+------+------+
|birth_year|weight|budget|height|
+----------+------+------+------+
|      1940|  44.0|  null|  1.53|
|      1990|  82.0|  null|  1.84|
|      1988| 

###Save the aggregated data in the Gold layer

In [0]:
from pyspark.sql.functions import avg

# Load Silver Table
df_silver = spark.read.format("delta").load("dbfs:/mnt/silver_table")

# Aggregate: Find avg budget per drink_level
df_gold = df_silver.groupBy("drink_level").agg(avg("weight").alias("avg_weight"))

# Save to Delta Lake (Gold Layer)
df_gold.write.format("delta").mode("overwrite").save("dbfs:/mnt/gold_table")

print("✅ Aggregated data stored in Gold layer")

✅ Aggregated data stored in Gold layer


###Lets Check Result

In [0]:
df_gold.printSchema()
df_gold.select("drink_level", "avg_weight").show(5)

root
 |-- drink_level: string (nullable = true)
 |-- avg_weight: double (nullable = true)

+--------------+-----------------+
|   drink_level|       avg_weight|
+--------------+-----------------+
|    abstemious|61.96078431372549|
|casual drinker|65.25531914893617|
|social drinker|           68.125|
+--------------+-----------------+



In [0]:
spark.read.format("delta").load("dbfs:/mnt/gold_table").show()

+--------------+-----------------+
|   drink_level|       avg_weight|
+--------------+-----------------+
|    abstemious|61.96078431372549|
|casual drinker|65.25531914893617|
|social drinker|           68.125|
+--------------+-----------------+



In [0]:
display(dbutils.fs.ls("/mnt"))

path,name,size,modificationTime
dbfs:/mnt/bronze_table/,bronze_table/,0,0
dbfs:/mnt/checkpoints/,checkpoints/,0,0
dbfs:/mnt/delta/,delta/,0,0
dbfs:/mnt/gold_table/,gold_table/,0,0
dbfs:/mnt/raw_data/,raw_data/,0,0
dbfs:/mnt/schema/,schema/,0,0
dbfs:/mnt/silver_table/,silver_table/,0,0


In [0]:
# Show Bronze Data
print("🔹 Bronze Data:")
spark.read.format("delta").load("dbfs:/mnt/bronze_table").show()




🔹 Bronze Data:
+------+---------+-----------+------+--------------+----------------+--------+---------+--------------+-----------+----------+------------+-------------------+---------+------------+------+------+------+------+-------------+
|userID| latitude|  longitude|smoker|   drink_level|dress_preference|ambience|transport|marital_status|      hijos|birth_year|    interest|        personality| religion|    activity| color|weight|budget|height|_rescued_data|
+------+---------+-----------+------+--------------+----------------+--------+---------+--------------+-----------+----------+------------+-------------------+---------+------------+------+------+------+------+-------------+
| U1001|22.139997|-100.978803| false|    abstemious|        informal|  family|  on foot|        single|independent|      1989|     variety|  thrifty-protector|     none|     student| black|    69|medium|  1.77|         null|
| U1002|22.150087|-100.983325| false|    abstemious|        informal|  family|   publ

In [0]:
# Show Silver Data
print("🔹 Silver Data:")
spark.read.format("delta").load("dbfs:/mnt/silver_table").show()



🔹 Silver Data:
+------+---------+-----------+------+--------------+----------------+--------+---------+--------------+-----------+----------+------------+-------------------+---------+------------+------+------+------+------+-------------+
|userID| latitude|  longitude|smoker|   drink_level|dress_preference|ambience|transport|marital_status|      hijos|birth_year|    interest|        personality| religion|    activity| color|weight|budget|height|_rescued_data|
+------+---------+-----------+------+--------------+----------------+--------+---------+--------------+-----------+----------+------------+-------------------+---------+------------+------+------+------+------+-------------+
| U1004|   18.867|    -99.183| false|    abstemious|        informal|  family|   public|        single|independent|      1940|     variety|        hard-worker|     none|professional| green|    44|medium|  1.53|         null|
| U1046|22.144415|-100.933097| false|social drinker|   no preference| friends|car own

In [0]:
# Show Gold Data
print("🔹 Gold Data:")
spark.read.format("delta").load("dbfs:/mnt/gold_table").show()

🔹 Gold Data:
+--------------+-----------------+
|   drink_level|       avg_weight|
+--------------+-----------------+
|    abstemious|61.96078431372549|
|casual drinker|65.25531914893617|
|social drinker|           68.125|
+--------------+-----------------+

